In [ ]:
import scanpy as sc
import scrublet as scr
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

Scrublet github tutorial

load in .mtx matrix just as the tutorial does

In [ ]:
# input_dir = 'C:/Users/julia/scRNAseq_analysis_project/samples_dir/BL_C/filtered_feature_bc_matrix'
# counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx').T.tocsc()
# genes = np.array(scr.load_genes(input_dir + '/features.tsv', delimiter='\t', column=1))

# print('Counts matrix shape: {} rows, {} columns'.format(counts_matrix.shape[0], counts_matrix.shape[1]))
# print('Number of genes in gene list: {}'.format(len(genes)))

load in as adata object and use adata.X

In [ ]:
file_path = 'C:/Users/julia/scRNAseq_analysis_project/samples_dir/BL_C/filtered_feature_bc_matrix'
adata = sc.read_10x_mtx(path=file_path, var_names='gene_symbols', cache=True)
adata.var_names_make_unique()

In [ ]:
OUTPUT_PATH = "C:/Users/julia/Project/scrublet_testing/scrublet_plots/scrublet_tutorial_adataobj_filtered"

Now add filtering for scrublet_tutorial_adataobj_filtered

In [ ]:
sc.pp.filter_cells(adata, min_genes=700)
sc.pp.filter_genes(adata, min_cells=3)
adata.uns['BL_C'] =  'BL_C'
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs.pct_counts_mt < 20, :]

In [ ]:
adata.X

In [ ]:
scrub = scr.Scrublet(adata.X, expected_doublet_rate=0.06)

In [ ]:
scrub._E_obs
#adata.X


In [ ]:
doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)

In [ ]:
scrub.call_doublets(threshold=0.23, verbose=True)

In [ ]:
scrub.plot_histogram();
plt.savefig(OUTPUT_PATH+'/histogram_BLC.png')


In [ ]:
scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))

In [ ]:
scrub.plot_embedding('UMAP', order_points=True);
plt.savefig(OUTPUT_PATH+'/UMAP_BLC.png')


Trying out my own thing

In [ ]:
file_path = 'C:/Users/julia/scRNAseq_analysis_project/samples_dir/BL_C/filtered_feature_bc_matrix'
adata = sc.read_10x_mtx(path=file_path, var_names='gene_symbols', cache=True)
adata.var_names_make_unique()

In [ ]:
OUTPUT_PATH = "C:/Users/julia/Project/scrublet_testing/scrublet_plots/pipeline_scrublet"

In [ ]:
sc.pp.filter_cells(adata, min_genes=700)
sc.pp.filter_genes(adata, min_cells=3)
adata.uns['BL_C'] =  'BL_C'
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs.pct_counts_mt < 20, :]

In [ ]:
adata.raw = adata

In [ ]:
adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4) # still included highly expressed data for now
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', subset=False) # EDIT: added this line for testing

In [ ]:
adata = adata[:, adata.var.highly_variable] # Actually do the slicing
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt']) # regress out sequencing depth and % MT-RNA
sc.pp.scale(adata) # moved scale under regressed out according to tutorials & chatgpt

In [ ]:
adata

In [ ]:
sc.tl.pca(adata, n_comps=50)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

In [ ]:
title='Cluster Leidenalg Neuron-Astrocyte Co-Culture'
sc.pl.umap(adata, color=['leiden'], title=title, legend_loc='on data', legend_fontsize=6, show=False)

Trying sc.external.pp.scrublet_simulate_doublets

In [ ]:
rawadata =  adata.raw.to_adata()

In [ ]:
rawadata.X

*Simulate my own doublets using scanpy*
(Does not actually do anything with the anndata object yet)
Simulate doublets by adding the counts of random observed transcriptome pairs.
I assumed I used the raw adata data, filtered for expression and variability (as for the scanpy documentation)

In [ ]:
adata_sim = sc.external.pp.scrublet_simulate_doublets(rawadata, layer=None, sim_doublet_ratio=2.0, synthetic_doublet_umi_subsampling=1.0, random_seed=0)

In [ ]:
adata_sim

*Call Scrublet according to the scanpy method, using the adata_sim*

In [ ]:
sc.external.pp.scrublet(rawadata, adata_sim=adata_sim)

In [ ]:
sc.external.pl.scrublet_score_distribution(rawadata)

Implement scrublet according to the book vignette found here: https://nbisweden.github.io/workshop-scRNAseq/labs/compiled/scanpy/scanpy_01_qc.html


In [ ]:
file_path = 'C:/Users/julia/scRNAseq_analysis_project/samples_dir/BL_C/filtered_feature_bc_matrix'
adata = sc.read_10x_mtx(path=file_path, var_names='gene_symbols', cache=True)
adata.var_names_make_unique()

In [ ]:
OUTPUT_PATH = "C:/Users/julia/Project/scrublet_testing/scrublet_plots/testing_scanpy"

In [ ]:
sc.pp.filter_cells(adata, min_genes=700)
sc.pp.filter_genes(adata, min_cells=3)
adata.uns['BL_C'] =  'BL_C'
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs.pct_counts_mt < 20, :]

In [ ]:
adata

simulate 2x the amount of doublets from the original (but filtered & unnormalized) dataset

In [ ]:
adata_sim = sc.external.pp.scrublet_simulate_doublets(adata, sim_doublet_ratio=2.0, synthetic_doublet_umi_subsampling=1.0)

In [ ]:
adata_sim

Detect doublets using the original dataset and the simulated doublets.

In [ ]:
sc.external.pp.scrublet(adata, adata_sim=adata_sim, expected_doublet_rate=0.06)

In [ ]:
adata

In [ ]:
adata.obs['predicted_doublet'].value_counts()

This is so wrong, no binomal division and finds waaaaayyy to many doublets

In [ ]:
sc.external.pl.scrublet_score_distribution(adata, show=False)
# plt.savefig(OUTPUT_PATH+'/scanpy_sim_doubl.png')

Run rest of the pipeline according to the script we already have

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4) # still included highly expressed data for now
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', subset=False) # EDIT: added this line for testing
adata = adata[:, adata.var.highly_variable] # Actually do the slicing
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt']) # regress out sequencing depth and % MT-RNA
sc.pp.scale(adata) # moved scale under regressed out according to tutorials & chatgpt

In [ ]:
sc.tl.pca(adata, n_comps=50)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

change predicted_doublet to str because otherwise the umap errors (this is a typeerror I couldn't figure out but it does what I want it to do now)

In [ ]:
adata.obs['doublet_info'] = adata.obs["predicted_doublet"].astype(str)

In [ ]:
sc.pl.umap(adata, color=['doublet_score', 'doublet_info'], show=False)
# plt.savefig(OUTPUT_PATH+'/UMAPscanpy_sim_doubl.png')

Simulating doublets manually with the scanpy function seems to not work that well. It does find a load of neotypical doublets but also loads of embedded ones I don't suspect are actual doublets (false positives) so I want to try this again but without simulating the doublets but just running ``sc.external.pp.scrublet()``

In [ ]:
file_path = 'C:/Users/julia/scRNAseq_analysis_project/samples_dir/BL_C/filtered_feature_bc_matrix'
adata = sc.read_10x_mtx(path=file_path, var_names='gene_symbols', cache=True)
adata.var_names_make_unique()

In [ ]:
OUTPUT_PATH = "C:/Users/julia/Project/scrublet_testing/scrublet_plots/scanpy_method_run_scrub"

In [ ]:
sc.pp.filter_cells(adata, min_genes=700)
sc.pp.filter_genes(adata, min_cells=3)
adata.uns['BL_C'] =  'BL_C'
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs.pct_counts_mt < 20, :]

It finds doublets, but ONLY if you adjust the threshold. SO adjust the treshold so it has a bimodal division clearly visible.

In [ ]:
sc.external.pp.scrublet(adata, adata_sim=None, batch_key=None, sim_doublet_ratio=2.0, expected_doublet_rate=0.06, stdev_doublet_rate=0.02, synthetic_doublet_umi_subsampling=1.0, knn_dist_metric='euclidean',
                         normalize_variance=True, log_transform=False, mean_center=True, n_prin_comps=30, use_approx_neighbors=True, get_doublet_neighbor_parents=False, n_neighbors=None, threshold=0.2,
                           verbose=True, copy=False, random_state=0)

In [ ]:
adata.obs['predicted_doublet'].value_counts()

In [ ]:
sc.external.pl.scrublet_score_distribution(adata, show=False)
plt.savefig(OUTPUT_PATH+'/scanpy_just_run_scrub_THRESHOLD_02.png')

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4) # still included highly expressed data for now
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', subset=False) # EDIT: added this line for testing
adata = adata[:, adata.var.highly_variable] # Actually do the slicing
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt']) # regress out sequencing depth and % MT-RNA
sc.pp.scale(adata) # moved scale under regressed out according to tutorials & chatgpt

In [ ]:
sc.tl.pca(adata, n_comps=50)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

In [ ]:
adata.obs['doublet_info'] = adata.obs["predicted_doublet"].astype(str)

In [ ]:
sc.pl.umap(adata, color=['doublet_score', 'doublet_info'], show=False)
plt.savefig(OUTPUT_PATH+'/UMAPscanpy_just_run_scrub_THRESHOLD_02.png')

we look at the umaps and we can see that the programm does find doublets or thinks that there are doublets, but the treshold is too high (this can happen as discribed in the paper and the scanpy API)

"see: threshold : Optional[float] (default: None)
Doublet score threshold for calling a transcriptome a doublet. If None, this is set automatically by looking for the minimum between the two modes of the doublet_scores_sim_ histogram. It is best practice to check the threshold visually using the doublet_scores_sim_ histogram and/or based on co-localization of predicted doublets in a 2-D embedding."

It appears that there are SOME neotypical doublets (two clusters at least?) so I decided to lower the treshold
threshold = 0.2 --> 554 doublets found

In [ ]:
pd.DataFrame(adata.obs).to_csv(OUTPUT_PATH+"/scrublet_calls.tsv",sep = '\t')

---------------------------------------------------------------------------------------------------

## Trying out scanpy again but with different samples ##

In [ ]:
file_path = 'C:/Users/julia/scRNAseq_analysis_project/samples_dir/BL_C/filtered_feature_bc_matrix'
adata = sc.read_10x_mtx(path=file_path, var_names='gene_symbols', cache=True)
adata.var_names_make_unique()

In [ ]:
OUTPUT_PATH = "C:/Users/julia/Project/scrublet_testing/scrublet_plots/Scanpy_testing/BL_C/"

if you run find doublets BEFORE slicing out all the mito genes it finds loads of doublets? I guess this kind of makes sense

In [ ]:
sc.pp.filter_cells(adata, min_genes=700)
sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs.pct_counts_mt < 20, :]

In [ ]:
adata

In [ ]:
adata.raw = adata

In [ ]:
adata = adata.raw.to_adata()

In [ ]:
adata

In [ ]:
sc.external.pp.scrublet(adata, adata_sim=None, batch_key=None, sim_doublet_ratio=2.0, expected_doublet_rate=0.06, stdev_doublet_rate=0.02, synthetic_doublet_umi_subsampling=1.0, knn_dist_metric='euclidean',
                         normalize_variance=True, log_transform=False, mean_center=True, n_prin_comps=30, use_approx_neighbors=True, get_doublet_neighbor_parents=False, n_neighbors=None, threshold=None,
                           verbose=True, copy=False, random_state=0)

In [ ]:
adata.obs['predicted_doublet'].value_counts()

In [ ]:
sc.external.pl.scrublet_score_distribution(adata, show=False)
plt.savefig(OUTPUT_PATH+'BL_C_scanpy_histo_THRESH_04.png')

In [ ]:
adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4) # still included highly expressed data for now
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', subset=False) # EDIT: added this line for testing
adata.raw = adata
adata = adata[:, adata.var.highly_variable] # Actually do the slicing
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt']) # regress out sequencing depth and % MT-RNA
sc.pp.scale(adata) # moved scale under regressed out according to tutorials & chatgpt

In [ ]:
sc.tl.pca(adata, n_comps=50)

In [ ]:
adata

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

In [ ]:
adata

In [ ]:
adata.obs['doublet_info'] = adata.obs["predicted_doublet"].astype(str)

In [ ]:
sc.pl.umap(adata, color=['doublet_score', 'doublet_info'], show=False)
plt.savefig(OUTPUT_PATH+'/BL_C_scanpy_UMAP_THRESH_04.png')

In [ ]:
adata = adata.raw.to_adata()
print(adata)
adata = adata[adata.obs['doublet_info'] == 'False',:]

In [ ]:
adata

then I could call back to the raw adata set, remove the doublets and do DEA.
- I would be saving the clustering, pca and HVG plot as they are, I am guessing they then still contain the doublets but I don't think this is an issue for they are not that many and might indicate a process.

In [ ]:
file_path = 'C:/Users/julia/scRNAseq_analysis_project/samples_dir/BL_N/filtered_feature_bc_matrix'
adata = sc.read_10x_mtx(path=file_path, var_names='gene_symbols', cache=True)
adata.var_names_make_unique()
adata